## song2vec

In [ ]:
#coding: utf-8
import multiprocessing
import gensim
import sys
from random import shuffle

#解析出一个歌单中歌曲list，返回的是经过多次shuffle后的歌曲list，每个list元素为一个歌曲list
def parse_playlist_get_sequence(in_line,playlist_sequence):
    song_sequence = []
    contents = in_line.strip().split("\t")
    for song in contents[1:]:
        try:
            song_id,song_name,artist,popularity = song.split(":::")
            song_sequence.append(song_id)
        except:
            print("song format error")
            print(song+"\n")
    for i in range(len(song_sequence)):
        shuffle(song_sequence)
        playlist_sequence.append(song_sequence)
        
#训练song2vec
def train_song2vec(in_file,out_file):
    playlist_sequence = []
    for line in open(in_file):
        parse_playlist_get_sequence(line,playlist_sequence)
    cores = multiprocessing.cpu_count() #统计cpu的个数
    print("using all"+str(cores)+"cores")
    print("training word2ve model...")
    model = gensim.models.Word2Vec(sentences=playlist_sequence,size=150,window=7,min_count=3,workers=cores) #只有当word出现min_count，才求其word2vec
    print("saving model...")
    model.save(out_file)
        
            

In [ ]:
song_sequence_file = "./popularity.playlist"
model_file = "./song2vec.model"
train_song2vec(song_sequence_file,model_file)

In [ ]:
import cPickle as pickle
song_dic = pickle.load(open("popularity_song.pkl","rb")) #????
model_str = "./song2vec.model"
model = gensim.models.Word2Vec.load(model_str) #下载song2vec model

In [ ]:
song_id_list = song_dic.keys()[1000:1500:50] #song_dic={key:id,value:name}
for song_id in song_id_list:
    result_song_list = model.most.similarity(song_id)
    print(song_id,song_dic[song_id])
    print("\n相似歌曲 和 相似度 分别为")
    for song in result_song_list:
        print("\t",song_dic[song[0],song[1]]) #相似歌曲，相似度
    print("\n")
    